## Environment Setup

In [6]:
!python -m spacy download en_core_web_trf
!python -m spacy download zh_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 37 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 417.4 MB 20 kB/s 
     |████████████████████████████████| 2.4 MB 18.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_trf')


In [3]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lo

In [4]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [5]:
!pip install spacy[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Training from Scratch

In [ ]:
import json
import csv

training_data = {'classes' : ["O","B-GEO","I-GEO","B-ORG","I-ORG","B-PER","I-PER","B-TIM","I-TIM","B-GPE","I-GPE"], 'annotations' : []}
with open('/content/drive/MyDrive/CSIT_Internship/ner.csv', 'r') as f:
    data = csv.reader(f)
    next(data)
    for lines in data:
      temp_dict={}
      temp_dict['entities']=[]
      temp_dict['text']=lines[0]
      idx=start=0
      wordlst=lines[0].split(" ")
      tagslist=lines[1].split(" ")
      for i in range(len(wordlst)):
        end=start+len(wordlst[i])-1
        if tagslist[i]!="O":
          #print(lines)
          label=tagslist[i].upper()
          temp_dict['entities'].append((start,end,label))
        start+=len(wordlst[i])+1
      #print(temp_dict)
      training_data['annotations'].append(temp_dict)

print(training_data['annotations'][2])


{'entities': [(20, 27, 'B-TIM'), (62, 68, 'B-GEO'), (97, 103, 'B-ORG'), (190, 194, 'B-GEO'), (196, 205, 'I-GEO')], 'text': 'Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .'}


In [ ]:
cleaned_data={'entities':[],'text':''}
for j in training_data['annotations']:
  new=[]
  for i in range (len(j['entities'])-1):
    if j['entities'][i+1][2][0]=="B": 
      if j['entities'][i][2][0]=="B": #i is B and i+1 is B
        start=j['entities'][i][0]
        end=j['entities'][i][1]
        label=j['entities'][i][2][2:]
        if label=="PER":
          label="PERSON"
        if label=="TIM":
          label="TIME"
        if label=="GEO":
          label="LOC"
        new.append((start,end,label))
      else: #i is I, i+1 is B
        start=prevStart
        end=j['entities'][i][1]
        label=j['entities'][i][2][2:]
        if label=="PER":
          label="PERSON"
        if label=="TIM":
          label="TIME"
        if label=="GEO":
          label="LOC"
        new.append((start,end,label))    
        #print(new)
    elif j['entities'][i+1][2][0]=="I":
      if j['entities'][i][2][0]=="B": #i is B, i+1 is I
        prevStart=j['entities'][i][0]
      else: #i is I, i+1 is I
        continue
    if (i+2)==len(j['entities']):
      if j['entities'][i+1][2][0]=="B":
        start=j['entities'][i+1][0]
        end=j['entities'][i+1][1]
        if label=="PER":
          label="PERSON"
        if label=="TIM":
          label="TIME"
        if label=="GEO":
          label="LOC"
        new.append((start,end,label))
      else:
        start=prevStart
        end=j['entities'][i+1][1]
        label=j['entities'][i+1][2][2:]
        if label=="PER":
          label="PERSON"
        if label=="TIM":
          label="TIME"
        if label=="GEO":
          label="LOC"
        new.append((start,end,label))
    #print(new)
  j['entities']=new

print(training_data['annotations'][2])

{'entities': [(20, 27, 'TIME'), (62, 68, 'LOC'), (97, 103, 'ORG'), (190, 205, 'LOC')], 'text': 'Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .'}


In [ ]:
TRAININ_DATA=[]
for i in training_data['annotations']:
  TRAININ_DATA.append((i['text'],{"entities":i['entities']}))

print(len(TRAININ_DATA))
TRAINING_DATA=TRAININ_DATA[0:10]
print(TRAINING_DATA[2])
print(len(TRAINING_DATA))

47959
('Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .', {'entities': [(20, 27, 'TIME'), (62, 68, 'GEO'), (97, 103, 'ORG'), (190, 205, 'GEO')]})
10


In [ ]:
print(TRAINING_DATA)

[('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .', {'entities': [(48, 53, 'GEO'), (111, 117, 'GPE'), (77, 80, 'GEO')]}), ('Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .', {'entities': [(0, 6, 'GPE'), (108, 111, 'ORG'), (87, 95, 'TIM')]}), ('Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .', {'entities': [(20, 27, 'TIM'), (62, 68, 'GEO'), (97, 103, 'ORG'), (100, 205, 'GEO')]}), ('They left after a tense hour-long standoff with riot police .', {'entities': []}), ('U.N. relief coordinator Jan Egeland said Sunday , U.S. , Indonesian and Australian military helicopters are ferrying out food and supplies to remote areas of western

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

  0%|          | 79/47959 [00:00<01:01, 773.54it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

  1%|          | 343/47959 [00:00<00:55, 850.48it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

  1%|          | 429/47959 [00:00<01:05, 724.35it/s]

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

100%|█████████▉| 47836/47959 [01:22<00:00, 669.35it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

100%|██████████| 47959/47959 [01:22<00:00, 579.94it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/CSIT_Internship/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 35 kB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 25.5 MB/s 
     |████████████████████████████████| 4.7 MB 73.6 MB/s 
     |████████████████████████████████| 120 kB 64.7 MB/s 
     |████████████████████████████████| 6.6 MB 62.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-31 07:28:57,967] [INFO] Set up nlp object from config
[2022-08-31 07:28:57,978] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-08-31 07:28:57,982] [INFO] Created vocabulary
[2022-08-31 07:29:01,140] [INFO] Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0xa2d14000 @  0x7f14415ea2a4 0x7f14362b1a7d 0x7f14362b0c5d 0x7f14362ad435 0x7f14362adbde 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x5127f1 0x7f12c1828218 0x7f12c182a1de 0x7f12c18310d7 0x7f12c18347df 0x594b72 0x515600 0x593dd7 0x5118f8 0x7f12c1828218 0x7f12c182a1de 0x7f12c1830db0 0x593835 0x548c51 0x5127f1 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x549e0e 0x593fce
tcmalloc: large alloc 1248116736 bytes == 0x7e0ac000 @  0x7f14415ea2a4 0x7f14362af193 0x7f14362b0c5d 0x7f14362ad435 0x7f14362adbde 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x548ae9 0x51566f 0x7f12c1828218 

In [ ]:
import spacy
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Thousands of demonstrators have marched through London to protest the war in Iraq and demand")
#doc = nlp_ner("This is a sentence.")
for token in doc:
    print(token.text, token.pos_, token.dep_)
#print([(w.text, w.pos_) for w in doc])

Thousands  
of  
demonstrators  
have  
marched  
through  
London  
to  
protest  
the  
war  
in  
Iraq  
and  
demand  


## Training Pre-Trained Spacy Model

In [ ]:
!pip install "spacy~=2.0.1"
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import random
import pathlib

iterations = 3
try:
    model = 'en_core_web_sm'
    nlp = spacy.load(model)  # load existing spacy model
except:
    model = None
    print("Exception")
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print("Create NER")
else:
    ner = nlp.get_pipe('ner')
    print("Existing NER")

# Add new entity labels to entity recognizer
for _, annotations in TRAINING_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(iterations):
        print("Starting iteration " + str(itn))
        random.shuffle(TRAINING_DATA)
        losses = {}
        for text, annotations in TRAINING_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

# Save model
output_dir = 'model_test5'
if output_dir is not None:
    output_dir = pathlib.Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = model  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

# Test the saved model
output_dir = 'model_test5'
print("Loading from", output_dir)

nlp2 = spacy.load('model_test5')
test_text = """
   Facebook has reached a preliminary agreement in a long-running lawsuit 
   seeking damages from the social network for allowing third parties, 
   including the company Cambridge Analytica, to access users' private data.
"""
doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

Existing NER
Statring iteration 0
{'ner': 104.13547344245755}
Statring iteration 1
{'ner': 94.55942755141612}
Statring iteration 2
{'ner': 75.17625636541823}
Saved model to model_test5
Loading from model_test5
ORDINAL third
FAC Cambridge Analytica


## Use of English and Chinese spacy models

Example text:

US interest rates hit 14-year high in inflation battle; Fed chief Powell vows to 'keep at it'

乌克兰两座核电站先后遭俄军轰炸 国际担心可能引发核灾难

(translation: Two Ukrainian nuclear power plants have been bombed by Russian forces, there is fear that this may lead to a nuclear disaster.)

In [1]:
!pip install spacy
!python -m spacy download en_core_web_trf
!python -m spacy download zh_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 35 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [7]:
import spacy

In [8]:
language = input("Language Option (Chinese or English) or end: ")
while language!="end":
  if language == "Chinese":
    nlp = spacy.load('zh_core_web_trf')
  elif language=="English":
    nlp = spacy.load('en_core_web_trf')
  text = input("Input text for NER: ")
  doc = nlp(text)
  from spacy import displacy
  displacy.render(doc, style="ent", jupyter=True)
  language = input("Language Option (Chinese or English) or end: ")

Language Option (Chinese or English) or end: English
Input text for NER: US interest rates hit 14-year high in inflation battle; Fed chief Powell vows to 'keep at it'


Language Option (Chinese or English) or end: Chinese
Input text for NER: 乌克兰两座核电站先后遭俄军轰炸 国际担心可能引发核灾难


Language Option (Chinese or English) or end: end
